# Tensorflow MNIST Model

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
 
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf

if __name__ == '__main__':
    
    x = tf.placeholder(tf.float32, [None,784], name="x")

    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x,W) + b, name="y")

    y_ = tf.placeholder(tf.float32, [None, 10])


    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()

    sess = tf.Session()
    sess.run(init)

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_:mnist.test.labels}))

    saver = tf.train.Saver()

    saver.save(sess, "model/deep_mnist_model")



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.9159


Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python36:0.6 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 deep-mnist:0.1

13b5515bb7074825056c4795cacd4348b6d28272def928380116a1fe9ea30659


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:
RECEIVED RESPONSE:
Success:True message:
Request:
data {
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.948
        }
        values {
          number_value: 0.333
        }
        values {
          number_value: 0.318
        }
        values {
          number_value: 0.966
        }
        values {
          number_value: 0.711
        }
        values {
          number_value: 0.982
        }
        values {
          number_value: 0.776
        }
        values {
          number_value: 0.638
        }
        values {
          number_value: 0.697
        }
        values {
          number_value: 0.714
        }
        values {
          number_value: 0.054
        }
        values {
          number_value: 0.689
        }
        values {
          number_value: 0.264
        }
        values {
          number_value: 0.7
        }
        values {
          number_value: 

In [5]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [7]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [8]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [9]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [10]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar 13 09:35:37 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     5s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Wed Mar 13 09:35:42 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Role
NAME          AGE
seldon-local  0s

==> v1/RoleBinding
NAME    AGE
s

In [11]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python2:0.6 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Looking in links: /whl
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Build completed successfully


In [12]:
!kubectl create -f deep_mnist.json

seldondeployment.machinelearning.seldon.io/deep-mnist created


In [14]:
!kubectl rollout status deploy/deep-mnist-single-model-8969cc0

Waiting for deployment "deep-mnist-single-model-8969cc0" rollout to finish: 0 of 1 updated replicas are available...
deployment "deep-mnist-single-model-8969cc0" successfully rolled out


In [15]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

RECEIVED RESPONSE:
Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.389
    values: 0.778
    values: 0.371
    values: 0.678
    values: 0.462
    values: 0.516
    values: 0.349
    values: 0.639
    values: 0.19
    values: 0.31
    values: 0.53
    values: 0.955
    values: 0.719
    values: 0.031
    values: 0.641
    values: 0.095
    values: 0.444
    values: 0.118
    values: 0.435
    values: 0.573
    values: 0.507
    values: 0.599
    values: 0.266
    values: 0.159
    values: 0.45
    values: 0.64
    values: 0.841
    values: 0.027
    values: 0.408
    values: 0.17
    values: 0.602
    values: 0.511
    values: 0.933
    values: 0.178
    values: 0.176
    values: 0.877
    values: 0.06
    values: 0.368
    values: 0.25
    values: 0.121
    values: 0.178
    values: 0.308
    values: 0.015
    values: 0.686
    values: 0.657
    values: 0.833
    values: 0.076
    values: 0.562
    values: 0.194
    values: 0.327
    values: 

In [16]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
